In [ ]:
print('Setup complete.')

# Constrained Tool Use - GPT-5-Mini Demo

**Focus**: plan constrained tool use; answer repo questions with ≤3 calls using GPT-5-Mini via AskSage

This notebook demonstrates how to efficiently use a limited set of tools to explore and understand codebases with GPT-5-Mini LLM calls via AskSage. We'll work with strict constraints to maximize the value of each tool call.

## Learning Objectives
- Plan tool usage strategically with call limits
- Use `list_dir`, `search_code`, and `read_text` effectively with GPT-5-Mini
- Answer repository questions within constraint budgets
- Develop efficient exploration patterns
- Handle safe byte caps and minimize API costs
- Leverage GPT-5-Mini's advanced reasoning capabilities

In [ ]:
# Install required packages
!pip install requests
!pip install asksageclient
!pip install pandas
!pip install numpy

import os
import pathlib
import json
import re
import requests
import pandas as pd
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass
from asksageclient import AskSageClient

print("✅ All packages installed and modules imported successfully!")

In [ ]:
# Function to load credentials from a JSON file
def load_credentials(filename):
    try:
        with open(filename) as file:
            return json.load(file)
    except FileNotFoundError:
        raise FileNotFoundError("The credentials file was not found.")
    except json.JSONDecodeError:
        raise ValueError("Failed to decode JSON from the credentials file.")

# Load the credentials
credentials = load_credentials('../../credentials.json')

# Extract the API key and email from the credentials
api_key = credentials['credentials']['api_key']
email = credentials['credentials']['Ask_sage_user_info']['username']

# Create an instance of the AskSageClient
ask_sage_client = AskSageClient(email, api_key)

print("✅ AskSage client initialized successfully!")

In [ ]:
# Verify GPT-5-Mini is available and set as default model
try:
    models_response = ask_sage_client.get_models()
    if 'response' in models_response:
        available_models = models_response['response']
        models_df = pd.DataFrame(available_models, columns=['Model Name'])
        print("📋 Available Models:")
        print(models_df)
        
        # Check if gpt-5-mini is available
        if 'gpt-5-mini' in available_models:
            print("\n✅ GPT-5-Mini is available!")
            DEFAULT_MODEL = 'gpt-5-mini'
        else:
            print("\n⚠️  GPT-5-Mini not found. Available alternatives:")
            gpt_models = [m for m in available_models if 'gpt' in m.lower()]
            for model in gpt_models[:5]:
                print(f"   - {model}")
            # Fallback to a similar model
            DEFAULT_MODEL = 'gpt-4o-mini' if 'gpt-4o-mini' in available_models else available_models[0]
            print(f"\n🔄 Using fallback model: {DEFAULT_MODEL}")
    else:
        print("❌ Unable to fetch models")
        DEFAULT_MODEL = 'gpt-5-mini'  # Assume it exists
        
except Exception as e:
    print(f"⚠️  Error checking models: {e}")
    DEFAULT_MODEL = 'gpt-5-mini'  # Assume it exists
    
print(f"\n🤖 Using model: {DEFAULT_MODEL}")

## Constraint Framework

### Available Tools
- `list_dir`: Explore directory structure using real file system
- `search_code`: Find specific patterns in code using GPT-5-Mini analysis
- `read_text`: Read file contents with GPT-5-Mini interpretation

### Constraints
- **Call Limit**: Maximum 3 tool calls per question
- **LLM Budget**: Efficient prompt usage to minimize API costs
- **Safe Byte Caps**: Limited file reading to prevent memory issues
- **Strategic Planning**: Each call must maximize information gain
- **GPT-5-Mini Optimization**: Leverage advanced reasoning for better insights

In [ ]:
@dataclass
class ToolCall:
    """Track tool usage for constraint compliance"""
    tool_name: str
    parameters: dict
    result_summary: str
    bytes_read: int = 0
    model_used: str = DEFAULT_MODEL

class GPT5MiniConstrainedExplorer:
    """GPT-5-Mini powered constrained tool explorer with AskSage"""
    
    def __init__(self, ask_sage_client, max_calls: int = 3, max_bytes_per_read: int = 5000, model: str = DEFAULT_MODEL):
        self.client = ask_sage_client
        self.max_calls = max_calls
        self.max_bytes_per_read = max_bytes_per_read
        self.model = model
        self.calls_made: List[ToolCall] = []
        
    def can_make_call(self) -> bool:
        return len(self.calls_made) < self.max_calls
    
    def _query_llm(self, prompt: str, system_prompt: str = None) -> str:
        """Query GPT-5-Mini via AskSage with optimized parameters"""
        try:
            response = self.client.query(
                user_input=prompt,
                model=self.model,
                system_prompt=system_prompt,
                temperature=0.1,  # Low temperature for consistent, focused responses
                max_tokens=2000   # Reasonable limit for cost control
            )
            
            if isinstance(response, dict) and 'response' in response:
                return response['response']
            elif isinstance(response, dict) and 'ret' in response:
                return response['ret']
            else:
                return str(response)
                
        except Exception as e:
            print(f"LLM query failed: {e}")
            return f"Error: {str(e)}"
    
    def list_dir(self, path: str) -> List[str]:
        """List directory contents using real file system"""
        if not self.can_make_call():
            raise Exception(f"Call limit exceeded: {self.max_calls}")
            
        try:
            result = os.listdir(path)
        except FileNotFoundError:
            result = []
        
        call = ToolCall(
            tool_name="list_dir",
            parameters={"path": path},
            result_summary=f"Found {len(result)} items: {', '.join(result[:3])}{'...' if len(result) > 3 else ''}",
            model_used=self.model
        )
        self.calls_made.append(call)
        
        print(f"🔍 Call {len(self.calls_made)}/{self.max_calls}: list_dir('{path}') using {self.model}")
        print(f"   Result: {result}")
        
        return result
    
    def search_code(self, pattern: str, path: str = "/content/sample_data") -> List[Tuple[str, int]]:
        """Search for code patterns using GPT-5-Mini analysis via AskSage"""
        if not self.can_make_call():
            raise Exception(f"Call limit exceeded: {self.max_calls}")
        
        # Get file list to search
        files_to_search = []
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith(('.py', '.txt', '.md', '.json', '.yaml', '.yml', '.csv')):
                    files_to_search.append(os.path.join(root, file))
        
        # Use GPT-5-Mini to analyze which files likely contain the pattern
        system_prompt = "You are a code analysis expert. Analyze file paths and predict which files likely contain specific patterns."
        
        prompt = f"""Given this search pattern: '{pattern}'
Files available: {files_to_search[:15]}

Analyze which files are most likely to contain code or content related to '{pattern}'.
Consider file names, extensions, and typical software project structures.
Return only the TOP 3 most relevant file paths, one per line, with no explanations."""
        
        llm_response = self._query_llm(prompt, system_prompt)
        relevant_files = [line.strip() for line in llm_response.split('\n') if line.strip() and ('/' in line.strip() or '\\' in line.strip())]
        
        if not relevant_files:
            relevant_files = files_to_search[:3]  # Fallback to first 3 files
        
        # Simple grep-like search in the identified files
        results = []
        for file_path in relevant_files[:5]:
            try:
                if os.path.exists(file_path):
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                        lines = f.readlines()
                        for i, line in enumerate(lines, 1):
                            if pattern.lower() in line.lower():
                                results.append((file_path, i))
            except Exception:
                continue
        
        call = ToolCall(
            tool_name="search_code",
            parameters={"pattern": pattern, "path": path},
            result_summary=f"GPT-5-Mini found {len(results)} matches for '{pattern}'",
            model_used=self.model
        )
        self.calls_made.append(call)
        
        print(f"🔍 Call {len(self.calls_made)}/{self.max_calls}: search_code('{pattern}', '{path}') using {self.model}")
        print(f"   Result: {results}")
        
        return results
        
    def read_text(self, file_path: str, max_bytes: Optional[int] = None) -> str:
        """Read file contents with GPT-5-Mini interpretation via AskSage"""
        if not self.can_make_call():
            raise Exception(f"Call limit exceeded: {self.max_calls}")
            
        max_bytes = max_bytes or self.max_bytes_per_read
        
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read(max_bytes)
        except Exception as e:
            content = f"Error reading file: {str(e)}"
        
        bytes_read = len(content)
        
        # Use GPT-5-Mini to analyze and summarize the content
        if len(content) > 100:
            system_prompt = "You are a technical document analyzer. Provide concise, accurate summaries focusing on key functionality and purpose."
            
            summary_prompt = f"""Analyze this file content and provide a comprehensive 2-3 sentence summary:

File: {file_path}
Content:
{content[:2000]}{'...(truncated)' if len(content) > 2000 else ''}

Focus on:
1. Main purpose and functionality
2. Key components or data structures
3. Notable patterns or technologies used

Provide only the summary, no preamble."""
            
            summary = self._query_llm(summary_prompt, system_prompt)
        else:
            summary = content
        
        call = ToolCall(
            tool_name="read_text",
            parameters={"file_path": file_path, "max_bytes": max_bytes},
            result_summary=f"GPT-5-Mini analyzed {bytes_read} bytes from {file_path}. Summary: {summary[:100]}...",
            bytes_read=bytes_read,
            model_used=self.model
        )
        self.calls_made.append(call)
        
        print(f"🔍 Call {len(self.calls_made)}/{self.max_calls}: read_text('{file_path}', max_bytes={max_bytes}) using {self.model}")
        print(f"   Result: {bytes_read} bytes read")
        print(f"   GPT-5-Mini Summary: {summary}")
        
        return content
    
    def get_call_summary(self) -> str:
        """Generate summary of all calls made"""
        summary = f"\n=== GPT-5-MINI CALL SUMMARY ({len(self.calls_made)}/{self.max_calls}) ===\n"
        for i, call in enumerate(self.calls_made, 1):
            summary += f"{i}. {call.tool_name} ({call.model_used}): {call.result_summary}\n"
        return summary

print("✅ GPT5MiniConstrainedExplorer class defined!")


## Demo: Strategic Repository Exploration with GPT-5-Mini

Let's demonstrate how to answer repository questions efficiently using our 3-call limit with GPT-5-Mini's advanced analysis capabilities.

In [ ]:
# Demo Question: "What files are in the sample data?"
print("🎯 QUESTION: What files are in the sample data?")
print("📋 STRATEGY: 1) List sample_data directory, 2) Search for interesting patterns, 3) Read a key file")
print(f"🤖 USING MODEL: {DEFAULT_MODEL}")
print()

explorer = GPT5MiniConstrainedExplorer(ask_sage_client, max_calls=3, model=DEFAULT_MODEL)

# Call 1: Strategic directory exploration
print("📍 CALL 1 STRATEGY: Explore sample_data directory structure")
sample_contents = explorer.list_dir("/content/sample_data")
print(f"✅ Found {len(sample_contents)} files/directories")
print()

# Call 2: GPT-5-Mini guided search for data patterns
print("📍 CALL 2 STRATEGY: Use GPT-5-Mini to search for data or CSV files")
data_matches = explorer.search_code("data")
print(f"✅ GPT-5-Mini found {len(data_matches)} data-related references")
print()

# Call 3: GPT-5-Mini analysis of the most interesting file
print("📍 CALL 3 STRATEGY: GPT-5-Mini analysis of a sample data file")
if sample_contents:
    target_file = f"/content/sample_data/{sample_contents[0]}"
    file_content = explorer.read_text(target_file, max_bytes=1000)
    print(f"✅ GPT-5-Mini successfully analyzed {target_file}")

print(explorer.get_call_summary())
print("\n🎉 EXPLORATION COMPLETE: Used GPT-5-Mini to efficiently analyze sample data within constraints")

## Demo: GPT-5-Mini Guided File Analysis

Now let's show how to find specific content using GPT-5-Mini's advanced reasoning capabilities.

In [ ]:
# Demo Question: "Find CSV files and analyze their structure using GPT-5-Mini"
print("🎯 QUESTION: Find CSV files and analyze their structure using GPT-5-Mini")
print("📋 STRATEGY: 1) GPT-5-Mini search for CSV patterns, 2) List directory for .csv files, 3) GPT-5-Mini analysis of CSV structure")
print(f"🤖 USING MODEL: {DEFAULT_MODEL}")
print()

explorer = GPT5MiniConstrainedExplorer(ask_sage_client, max_calls=3, model=DEFAULT_MODEL)

# Call 1: GPT-5-Mini guided search for CSV patterns
print("📍 CALL 1 STRATEGY: Use GPT-5-Mini to search codebase for CSV-related content")
csv_matches = explorer.search_code("csv")
print(f"✅ GPT-5-Mini found CSV references in {len(set(match[0] for match in csv_matches))} files")
print()

# Call 2: Explore sample data directory
print("📍 CALL 2 STRATEGY: List files in sample_data directory")
data_files = explorer.list_dir("/content/sample_data")
csv_files = [f for f in data_files if f.endswith('.csv')]
print(f"✅ Found {len(csv_files)} CSV files: {csv_files}")
print()

# Call 3: GPT-5-Mini analysis of CSV file structure
print("📍 CALL 3 STRATEGY: GPT-5-Mini deep analysis of CSV file structure")
if csv_files:
    csv_path = f"/content/sample_data/{csv_files[0]}"
    csv_content = explorer.read_text(csv_path, max_bytes=500)
    print(f"✅ GPT-5-Mini successfully analyzed {csv_files[0]} structure")
else:
    # Fallback to any available file
    if data_files:
        fallback_file = f"/content/sample_data/{data_files[0]}"
        content = explorer.read_text(fallback_file, max_bytes=500)
        print(f"✅ GPT-5-Mini analyzed available file: {data_files[0]}")

print(explorer.get_call_summary())
print("\n🎉 ANALYSIS COMPLETE: GPT-5-Mini guided us to efficiently find and analyze data files")

## Advanced GPT-5-Mini Strategic Planning Framework

### Key Advantages of GPT-5-Mini for Constrained Tool Use

1. **Enhanced Reasoning**: GPT-5-Mini's advanced reasoning helps predict most relevant files/locations
2. **Pattern Recognition**: Superior pattern matching for complex code analysis
3. **Context Understanding**: Better comprehension of file relationships and project structure
4. **Efficient Summarization**: More accurate and concise file content analysis
5. **Cost Optimization**: Balanced performance and API efficiency
6. **Multi-step Planning**: Enhanced ability to plan complex exploration strategies

In [ ]:
def gpt5_mini_plan_exploration_strategy(question: str, ask_sage_client, max_calls: int = 3, model: str = DEFAULT_MODEL) -> List[str]:
    """Generate GPT-5-Mini guided strategic plan for repository exploration"""
    
    system_prompt = "You are an expert software architect and code analyst. Create optimal exploration strategies for repository analysis within strict constraints."
    
    planning_prompt = f"""Given this repository analysis question: "{question}"

Available tools:
- list_dir(path): Lists files and directories
- search_code(pattern, path): Searches for patterns in code files using AI analysis
- read_text(file_path, max_bytes): Reads and analyzes file contents with AI interpretation

CONSTRAINTS: You have exactly {max_calls} tool calls to answer this question efficiently.

Create an optimal 3-step exploration strategy. Each step should specify:
1. Tool to use
2. Specific parameters
3. Expected outcome and information gain
4. How it builds toward answering the question

Prioritize steps that maximize information gain and build upon each other.

Format as: "Step N: tool_name(specific_parameters) - expected outcome and strategic value"
Return only the 3 steps, one per line."""
    
    try:
        response = ask_sage_client.query(
            user_input=planning_prompt,
            model=model,
            system_prompt=system_prompt,
            temperature=0.1,
            max_tokens=1000
        )
        
        # Extract response text
        if isinstance(response, dict) and 'response' in response:
            response_text = response['response']
        elif isinstance(response, dict) and 'ret' in response:
            response_text = response['ret']
        else:
            response_text = str(response)
            
        strategy = [line.strip() for line in response_text.split('\n') if line.strip() and 'Step' in line]
        return strategy[:3]  # Ensure exactly 3 steps
        
    except Exception as e:
        print(f"GPT-5-Mini planning failed: {e}")
        # Fallback strategy
        return [
            "Step 1: list_dir(/content/sample_data) - Understand directory structure and file types",
            "Step 2: search_code(relevant_terms) - Find relevant code patterns using AI analysis", 
            "Step 3: read_text(target_file) - Examine specific implementation with AI interpretation"
        ]

# Test the GPT-5-Mini planning function
questions = [
    "What data formats and file types are available in the sample data?",
    "Find any machine learning, AI, or data science related files and analyze their purpose", 
    "What is the structure and content of the largest or most complex file in the repository?",
    "Identify configuration files and understand the project's technology stack"
]

for q in questions:
    print(f"\n❓ Question: {q}")
    strategy = gpt5_mini_plan_exploration_strategy(q, ask_sage_client, model=DEFAULT_MODEL)
    print(f"📋 GPT-5-Mini Generated Strategy (using {DEFAULT_MODEL}):")
    for step in strategy:
        print(f"   {step}")

print("\n✅ GPT-5-Mini guided strategic planning framework complete!")

## Token Usage and Cost Monitoring

Monitor GPT-5-Mini usage to optimize costs while maintaining quality.

In [ ]:
def monitor_gpt5_mini_usage(ask_sage_client):
    """Monitor GPT-5-Mini token usage and provide cost insights"""
    
    try:
        # Get monthly token count
        token_response = ask_sage_client.count_monthly_tokens()
        
        if isinstance(token_response, dict) and 'response' in token_response:
            token_data = token_response['response']
            
            print("🔍 GPT-5-Mini Usage Report:")
            print("="*50)
            
            if isinstance(token_data, dict):
                for key, value in token_data.items():
                    print(f"{key}: {value:,} tokens")
            else:
                print(f"Total tokens used: {token_data:,}")
                
            print("="*50)
            
            # Efficiency tips for GPT-5-Mini
            print("\n💡 GPT-5-Mini Optimization Tips:")
            print("• Use temperature=0.1 for consistent, focused responses")
            print("• Set appropriate max_tokens limits (1000-2000 for analysis)")
            print("• Leverage system prompts for better context efficiency")
            print("• Batch similar queries when possible")
            print("• Use specific, targeted prompts for better cost per insight")
            
        else:
            print("Unable to retrieve token usage data")
            
    except Exception as e:
        print(f"Error monitoring usage: {e}")

# Run usage monitoring
monitor_gpt5_mini_usage(ask_sage_client)

## Best Practices Summary for GPT-5-Mini

### Maximizing GPT-5-Mini Information Gain
- **AI-Guided Discovery**: Leverage GPT-5-Mini's reasoning to predict most relevant files before exploring
- **Intelligent Filtering**: Use advanced context understanding to prioritize which files to examine
- **Smart Summarization**: Leverage superior language understanding for extracting key insights from large files
- **Multi-step Reasoning**: Utilize enhanced planning capabilities for complex exploration tasks

### Managing GPT-5-Mini and System Constraints
- **Call Budgeting**: Plan all 3 calls strategically, considering GPT-5-Mini's enhanced capabilities
- **Prompt Efficiency**: Use targeted, specific prompts optimized for GPT-5-Mini's reasoning abilities
- **Temperature Control**: Use low temperatures (0.1) for consistent, focused analytical outputs
- **Token Management**: Balance quality insights with cost efficiency

### Real-World Implementation
- **AskSage Integration**: Seamless integration with enterprise-grade AI infrastructure
- **File Type Awareness**: Enhanced understanding of various file formats (.py, .csv, .txt, .json, .md)
- **Error Handling**: Robust operations with intelligent fallback strategies
- **Scalability**: Optimized for both small projects and large codebases

This framework enables efficient repository exploration within tight constraints while leveraging GPT-5-Mini's advanced reasoning capabilities to maximize the value and accuracy of each tool call.